In [ ]:
sudo apt install ffmpeg
pip install torch, transformers, jiwer

In [37]:
import requests
import os
import boto3
import uuid
import time
import string
import jiwer
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import re


FILE_LOCAL = "HaTankistiot_n12-mp3.mp3"

# Initialize boto3 clients
transcribe_client = boto3.client("transcribe")
s3_client = boto3.client("s3")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32


def whisper_v3(audio_file):
    model_id = "openai/whisper-large-v3"
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id,
        torch_dtype=torch_dtype,
        use_safetensors=True
    )
    model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        chunk_length_s=30,
        batch_size=16,
        torch_dtype=torch_dtype,
        device=device,
    )

    prediction = pipe(audio_file, generate_kwargs={"language": "hebrew"})["text"]

    return prediction


def whisper_v2(audio_file):

    pipe = pipeline(
        "automatic-speech-recognition",
        model="openai/whisper-large-v2",
        chunk_length_s=30,
        batch_size=8,
        device=device,
    )

    prediction = pipe(audio_file, generate_kwargs={"language": "hebrew"})["text"]

    return prediction


def whisper_ivrit(audio_file):
    model_id = "ivrit-ai/whisper-large-v2-tuned"
    model = WhisperForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch_dtype,
        use_safetensors=True
    )
    model.to(device)
    processor = WhisperProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        chunk_length_s=10,
        stride_length_s=(4, 2),
        batch_size=8,
        torch_dtype=torch_dtype,
        device=device,
    )

    result = pipe(audio_file, generate_kwargs={"language": "hebrew"})["text"]

    return result


def amazon_transcribe(file_uri):
    bucket_name = "virginia-transcribe"
    object_key = file_uri
    s3_client.upload_file(file_uri, bucket_name, object_key)

    job_name = str(uuid.uuid4())
    media = file_uri.split(".")[-1]
    file_uri = f's3://{bucket_name}/{object_key}'

    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={"MediaFileUri": file_uri},
        MediaFormat=media,
        LanguageCode="he-IL",
    )

    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job["TranscriptionJob"]["TranscriptionJobStatus"]
        if job_status in ["COMPLETED", "FAILED"]:
            print(f"Job {job_name} is {job_status}.")
            if job_status == "COMPLETED":
                break;
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)

    json_url = job['TranscriptionJob']['Transcript']['TranscriptFileUri']
    file_content = requests.get(json_url)
    json_data = file_content.json()

    return json_data['results']['transcripts'][0]['transcript']


def remove_punctuation(text):
    # Create a translation table that maps punctuation to None (to remove them)
    # but maps the hyphen ('-') to a space (' ')
    translator = str.maketrans('', '', string.punctuation.replace('-', ''))
    translator[ord('-')] = ' '  # Map the hyphen to a space

    # Remove punctuation and replace hyphens with spaces
    text = text.translate(translator)

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    return text

In [25]:
transcribe_result = amazon_transcribe(FILE_LOCAL)
print(f"############### Amazon Transcribe ###############\n{transcribe_result}\n")

Waiting for aed22586-d182-4923-adf0-1d1e9d9e286f. Current status is IN_PROGRESS.
Waiting for aed22586-d182-4923-adf0-1d1e9d9e286f. Current status is IN_PROGRESS.
Waiting for aed22586-d182-4923-adf0-1d1e9d9e286f. Current status is IN_PROGRESS.
Job aed22586-d182-4923-adf0-1d1e9d9e286f is COMPLETED.
############### Amazon Transcribe ###############
להן הגר, הילה, טל שרה, מיכל, קרני, אופיר ותמר. רק בנות עשרים פלוס-מינוס, אבל הטנקיסטיות האלו והחברות שלהן לפלוגה, עשו ב שבע באוקטובר היסטוריה. הנשים הראשונות בישראל, כנראה בעולם כולו, שלחנו קמו בקרב שריון, קרב הרואי שבו הרגו עשרות מחבלים והצילו קיבוץ שלם. בצילום הזה, שמתפרסם כאן לראשונה, רואים את הטנק של קרני, מפקד את הפלוגה, פורץ בשערה את השער של קיבוץ חולית בדרכו לטהר את הקיבוץ במחבלים. יום הקרב שלהן מתחיל בבוקר של שמחת תורה, שבע באוקטובר. כשהמחבלים פורצים את הגדר מעזה לעוטף, הטנקיסטיות נמצאות בכלל הרחק משם, בגזרה אחרת, בגבול מצרים, בבסיס שלהן, בניצנה. הן מתחילות לנסוע עם שני כלים טנק וקטלן, סוג של רכב משוריין לכיוון צפון. הטנק דוהר באופן מאו

In [26]:
whisper_v3_result = whisper_v3(FILE_LOCAL)
print(f"############### Whisper V3 ###############\n{whisper_v3_result}\n")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


############### Whisper V3 ###############
 הגר, הילה, טל שרה, מיכל, קרני, אופיר ותמר רק בנות 20 פלוס מינוס אבל הטנקיסטיות האלו והחברות שלהן לפלוגה עשו בשבעה באוקטובר היסטוריה הנשים הראשונות בישראל כנראה בעולם כולו שלחמו בקרב שריון קרב הרוי שבו הרגו עשרות מחבלים והצילו קיבוץ שלם בצילום הזה שמתפרסם כאן לראשונה רואים את הטנק של קרני מפקד את הפלוגה פורץ בשערה את השערה של קיבוץ שלם. בצילום הזה שמתפרסם כאן לראשונה, רואים את הטנק של קארני, מפקד את הפלוגה, פורץ בשערה את השער של קיבוץ חולית, בדרכו לתהר את הקיבוץ ממחבלים. יום הקרב שלהן מתחיל בבוקר של שמחת תורה, 7 באוקטובר. כשהמחבלים פורצים את הגדר מאז על העוטף, הטנקיסטיות נמצאות בכלל הרחק משם, בגזרה אחרת, בגבול מצרים, בבסיס שלהן, בניצנה. הן מתחילות לנסוע עם שני כלים, טנק וקטלן, סוג של רכב משוריין, תודה רבה ומכר הלוחמות ממשיכות לקיבוץ חולית במקביל מיכל מפקדת המחלקה יוצאת עם הטנק שלה לסייע בקרב לקיבוץ סופה במשך שש שעות הן נלחמות לצד לוחמי לוטר אילת, שלדג וסיירת גולני פוגעות בעשרות מחבלים מנקות את הקיבוצים מרוצחים בעצם בשעה שש וחצי בבוקר טל מגיעה 

In [48]:
whisper_v2_result = whisper_v2(FILE_LOCAL)
print(f"############### Whisper V2 ###############\n{whisper_v2_result}\n")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


############### Whisper V2 ###############
 הגר, הילה, טל שרה, מיכל, קרני, אופיר ותמר רק בנות 20, פלוס מינוס אבל הטנקיסטיות האלו והחברות שלהן לפלוגה עשו בשבעה באוקטובר היסטוריה האנשים הראשונות בישראל, כנראה בעולם כולו שלחמו בקרב שריון קרב הרוי שבו הרגו עשרות מחבלים והצילו קיבוץ שלם בצילום הזה שמתפרסם כאן לראשונה רואים את הטנק של קרני מפקד את הפלוגה פורץ בשערה את השערה של קיבוץ שלם. בצילום הזה, שמתפרסם כאן לראשונה, רואים את הטנק של קרני, מפקדת הפלוגה, פורץ בסערה את השערה של קיבוץ חולית בדרכו לטהר את הקיבוץ ממחבלים. יום הקרב שלהם מתחיל בבוקר של שמחת תורה, 7 באוקטובר. כשהמחבלים פורצים את הגדר מאז על האוטף, הטנקיסטיות נמצאות בכלל הרחק משם, בגזרה אחרת, בגבול מצרים, בבסיס שלהם, בניצנה. הן מתחילות לנסוע עם שני כלים, טנק וקטלן, סוג של רכב משוריין, לכיוון צפון. הטנק דוהר באופן מאוד יוצא דופן, עשרות קילומטרים, על כביש 232, בדרך לישובי עוטף עזה. ליד מוצב סופה הן נתקלות בעשרות מחבלים ששועטים לעברן ומתקדמות לכיוון הגדר. בגדר המערכת של רצועת עזה הן מבחינות באותו פתח ענק שיצרו המחבלים ומאשירות שם טנק

In [52]:
whisper_ivrit_result = whisper_ivrit(FILE_LOCAL)
print(f"############### Whisper Ivrit ###############\n{whisper_ivrit_result}\n")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


############### Whisper Ivrit ###############
הם נכון להם הגר, הילה, טל סרה, מיכל, קרני, אופיר ותמר, רק בנות 20 פלוס מינוס, אבל הטנקיסטיות האלו והחברות שלהן לפלוגה עשו בשבעה באוקטובר היסטוריה. הנשים הראשונות בישראל, כנראה בעולם כולו, שלחמו בקרב שריון. קרב הרוי שבו הרגו עשרות מחבלים ויצילו קיבוץ שלם בצילום הזה שמתפרסם כאן לראשונה, רואים את הטנק של קרני, מפקד את הפלוגה, פורץ בשערה את השער של קיבוץ חולית בדרכו לתהר את הקיבוץ ממחבלים יום הקרב שלהם מתחיל בבוקר של שמחת תורה, 7 באוקטובר. כשהמחבלים פורצים את הגדר מאז על האוטף, הטנקיסטיות נמצאות בכלל הרחק משם, בגזרה אחרת, בגבול מצרים, בבסיס שלהן, בניצנה הן מתחילות לנסוע עם שני כלים, טאנק וקטלן, סוג של רכב משוריין, לכיוון צפון. הטא�נק דוהר באופן מאוד יוצא דופן עשרות קילומטרים על כביש 232 בדרך לישובי עוטף עזה ליד מוצב סופה הן נתקלות בעשרות מחבלים ששואטים לעברן ומתקדמות לכיוון הגדר. בגדר המערכת של רצועת עזה הן מבחינות באותו פתח ענק שיצרו המחבלים ומשאירות שם טנק אחד לשמור, הטנק של מפקדת המחלקה שקד, שמונע מעכשיו מאזתים להמשיך ולעבור דרכו, לכיוון עזה

In [53]:
if 'transcribe_result' in globals():
    print(f"############### Amazon Transcribe ###############\n{transcribe_result}\n")
if 'whisper_v3_result' in globals():
    print(f"############### Whisper V3 ###############\n{whisper_v3_result}\n")
if 'whisper_v2_result' in globals():
    print(f"############### Whisper V2 ###############\n{whisper_v2_result}\n")
if 'whisper_ivrit_result' in globals():
    print(f"############### Whisper Ivrit ###############\n{whisper_ivrit_result}\n")

############### Amazon Transcribe ###############
להן הגר, הילה, טל שרה, מיכל, קרני, אופיר ותמר. רק בנות עשרים פלוס-מינוס, אבל הטנקיסטיות האלו והחברות שלהן לפלוגה, עשו ב שבע באוקטובר היסטוריה. הנשים הראשונות בישראל, כנראה בעולם כולו, שלחנו קמו בקרב שריון, קרב הרואי שבו הרגו עשרות מחבלים והצילו קיבוץ שלם. בצילום הזה, שמתפרסם כאן לראשונה, רואים את הטנק של קרני, מפקד את הפלוגה, פורץ בשערה את השער של קיבוץ חולית בדרכו לטהר את הקיבוץ במחבלים. יום הקרב שלהן מתחיל בבוקר של שמחת תורה, שבע באוקטובר. כשהמחבלים פורצים את הגדר מעזה לעוטף, הטנקיסטיות נמצאות בכלל הרחק משם, בגזרה אחרת, בגבול מצרים, בבסיס שלהן, בניצנה. הן מתחילות לנסוע עם שני כלים טנק וקטלן, סוג של רכב משוריין לכיוון צפון. הטנק דוהר באופן מאוד יוצא דופן, עשרות קילומטרים, על כביש שתים מאות שלשים ושתים בדרך ליישובי עוטף עזה. ליד מוצב סופה, הן נתקלות בעשרות מחבלים ששועטים לעברן ומתקדמות לכיוון הגדר. בגדר המערכת של רצועת עזה הן מבחינות באותו פתח ענק שיצרו המחבלים, ומשאירות שם טנק אחד לשמור, הטנק של מפקדת המחלקה שקד, שמונע מעכשיו מהזתים לה

In [54]:
if 'transcribe_result' in globals():
    clean_transcribe = remove_punctuation(transcribe_result)
    print(f"############### Amazon Transcribe Clean ###############\n{clean_transcribe}\n")
if 'whisper_v3_result' in globals():
    clean_whisper_v3 = remove_punctuation(whisper_v3_result)
    print(f"############### Whisper V3 Clean ###############\n{clean_whisper_v3}\n")
if 'whisper_v2_result' in globals():
    clean_whisper_v2 = remove_punctuation(whisper_v2_result)
    print(f"############### Whisper V2 Clean ###############\n{clean_whisper_v2}\n")
if 'whisper_ivrit_result' in globals():
    clean_whisper_ivrit = remove_punctuation(whisper_ivrit_result)
    print(f"############### Whisper Ivrit Clean ###############\n{clean_whisper_ivrit}\n")

reference = open("HaTankistiot_n12_refrence.txt", "r").read()
clean_refrence = remove_punctuation(reference)
print(f"############### Refrence Clean ###############\n{clean_refrence}\n")

############### Amazon Transcribe Clean ###############
להן הגר הילה טל שרה מיכל קרני אופיר ותמר רק בנות עשרים פלוס מינוס אבל הטנקיסטיות האלו והחברות שלהן לפלוגה עשו ב שבע באוקטובר היסטוריה הנשים הראשונות בישראל כנראה בעולם כולו שלחנו קמו בקרב שריון קרב הרואי שבו הרגו עשרות מחבלים והצילו קיבוץ שלם בצילום הזה שמתפרסם כאן לראשונה רואים את הטנק של קרני מפקד את הפלוגה פורץ בשערה את השער של קיבוץ חולית בדרכו לטהר את הקיבוץ במחבלים יום הקרב שלהן מתחיל בבוקר של שמחת תורה שבע באוקטובר כשהמחבלים פורצים את הגדר מעזה לעוטף הטנקיסטיות נמצאות בכלל הרחק משם בגזרה אחרת בגבול מצרים בבסיס שלהן בניצנה הן מתחילות לנסוע עם שני כלים טנק וקטלן סוג של רכב משוריין לכיוון צפון הטנק דוהר באופן מאוד יוצא דופן עשרות קילומטרים על כביש שתים מאות שלשים ושתים בדרך ליישובי עוטף עזה ליד מוצב סופה הן נתקלות בעשרות מחבלים ששועטים לעברן ומתקדמות לכיוון הגדר בגדר המערכת של רצועת עזה הן מבחינות באותו פתח ענק שיצרו המחבלים ומשאירות שם טנק אחד לשמור הטנק של מפקדת המחלקה שקד שמונע מעכשיו מהזתים להמשיך ולעבור דרכו לכיוון עזה או

In [55]:
if 'transcribe_result' in globals():
    error_transcribe = jiwer.process_words(clean_refrence, clean_transcribe)
    print(f"""############### Amazon Transcribe Error ###############
    WER (Word Error Rate) [lower is better] : {error_transcribe.wer * 100:.2f}%
    WIL (Word Information Lost) [lower is better] : {error_transcribe.wil * 100:.2f}%
    """)
if 'whisper_v3_result' in globals():
    error_whisper_v3 = jiwer.process_words(clean_refrence, clean_whisper_v3)
    print(f"""############### Whisper V3 Error ###############
    WER (Word Error Rate) [lower is better] : {error_whisper_v3.wer * 100:.2f}%
    WIL (Word Information Lost) [lower is better] : {error_whisper_v3.wil * 100:.2f}%
    """)
if 'whisper_v2_result' in globals():
    error_whisper_v2 = jiwer.process_words(clean_refrence, clean_whisper_v2)
    print(f"""############### Whisper V2 Error ###############
    WER (Word Error Rate) [lower is better] : {error_whisper_v2.wer * 100:.2f}%
    WIL (Word Information Lost) [lower is better] : {error_whisper_v2.wil * 100:.2f}%
    """)
if 'whisper_ivrit_result' in globals():
    error_whisper_ivrit = jiwer.process_words(clean_refrence, clean_whisper_ivrit)
    print(f"""############### Whisper Ivrit Error ###############
    WER (Word Error Rate) [lower is better] : {error_whisper_ivrit.wer * 100:.2f}%
    WIL (Word Information Lost) [lower is better] : {error_whisper_ivrit.wil * 100:.2f}%
    """)

############### Amazon Transcribe Error ###############
    WER (Word Error Rate) [lower is better] : 11.19%
    WIL (Word Information Lost) [lower is better] : 18.67%
    
############### Whisper V3 Error ###############
    WER (Word Error Rate) [lower is better] : 39.07%
    WIL (Word Information Lost) [lower is better] : 48.39%
    
############### Whisper V2 Error ###############
    WER (Word Error Rate) [lower is better] : 41.07%
    WIL (Word Information Lost) [lower is better] : 51.04%
    
############### Whisper Ivrit Error ###############
    WER (Word Error Rate) [lower is better] : 19.87%
    WIL (Word Information Lost) [lower is better] : 31.71%
    
